In [ ]:
from google.colab import drive
import sys

# Mount Google Drive
drive.mount('/content/drive')

abs_path = '/content/drive/My Drive/Deep Learning Midterm/'
data_path = '/content/drive/My Drive/Deep Learning Midterm/data/'

import os
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Bidirectional
from tensorflow.keras.metrics import MeanSquaredError
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
import tensorflow as tf

import matplotlib.pylab as pylab
params = {'legend.fontsize': 'large',
          'figure.titlesize': 'large',
          'figure.figsize': (11, 8),
         'axes.labelsize': 'large',
         'axes.titlesize':'large',
         'xtick.labelsize':'large',
         'ytick.labelsize':'large'}
pylab.rcParams.update(params)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# How Well Can Deep NN's Predict Crypto Prices?
## Predicting a minor coin (Shiba Inu) using other coins
**Overview:** In this project we will attempt to use some of the most popular coins like Bitcoin, Ether, and Cardano to predict the price of a relatively new, trendy, coin: Shiba Inu. For this project we will collect high-frequency trading patterns for the coins and see if Neural Networks can outpredict the simple forecasting methods. 

### Data Collection

In [ ]:
#### Crypto currency prices via API
!pip install yfinance
import yfinance as yf

In [ ]:
# save crypto prices in Google Drive
tickers = ["BTC-USD", "ETH-USD", "ATOM-USD", "DOGE-USD", "ADA-USD", "XRP-USD", "SHIB-USD"]
tickers  = tickers
for ticker in tickers:
  crypto_data = yf.download(tickers=ticker, period='1mo', interval="2m", group_by='ticker', threads=True)
  crypto_data.to_csv(os.path.join(data_path, "coin_"+ ticker + ".csv"))


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [ ]:
os.listdir(data_path)

['coin_BTC-USD.csv',
 'coin_ETH-USD.csv',
 'coin_ATOM-USD.csv',
 'coin_DOGE-USD.csv',
 'coin_ADA-USD.csv',
 'coin_XRP-USD.csv',
 'coin_SHIB-USD.csv']

In [ ]:
# Filtering out the data and handling Datetimes
coin_dataframes = dict()

def date_format(date_str):
    return pd.to_datetime(date_str)


for fn in os.listdir(data_path):
    coin_name = fn.split("_")[1][:-4]
    df = pd.read_csv(os.path.join(data_path, fn), parse_dates=['Datetime'], dayfirst=True)
    coin_dataframes[coin_name] = df.sort_values('Datetime')
    df['Datetime'] = pd.to_datetime(df['Datetime'], utc=True)
    print(df['Datetime'].dtype)

coin_dataframes['SHIB-USD'].tail()

datetime64[ns, UTC]
datetime64[ns, UTC]
datetime64[ns, UTC]
datetime64[ns, UTC]
datetime64[ns, UTC]
datetime64[ns, UTC]
datetime64[ns, UTC]


,Datetime,Open,High,Low,Close,Adj Close,Volume
20066,2022-03-22 09:18:00+00:00,0.000024,0.000024,0.000024,0.000024,0.000024,397440
20067,2022-03-22 09:20:00+00:00,0.000024,0.000024,0.000024,0.000024,0.000024,136640
20068,2022-03-22 09:22:00+00:00,0.000024,0.000024,0.000024,0.000024,0.000024,6720
20069,2022-03-22 09:24:00+00:00,0.000024,0.000024,0.000024,0.000024,0.000024,745792
20070,2022-03-22 09:26:00+00:00,0.000024,0.000024,0.000024,0.000024,0.000024,0


In [ ]:
coin_names = []
for fn in os.listdir(data_path):
    coin_name = fn.split("_")[1][:-4]
    coin_names.append(coin_name)

for name in coin_names:
  print(name, coin_dataframes[name]['Datetime'].min(),
        coin_dataframes[name]['Datetime'].max(), coin_dataframes[name].shape)


BTC-USD 2022-02-22 09:28:00+00:00 2022-03-22 09:26:00+00:00 (19966, 7)
ETH-USD 2022-02-22 09:28:00+00:00 2022-03-22 09:26:00+00:00 (20101, 7)
ATOM-USD 2022-02-22 09:28:00+00:00 2022-03-22 09:26:00+00:00 (20103, 7)
DOGE-USD 2022-02-22 09:28:00+00:00 2022-03-22 09:26:00+00:00 (20042, 7)
ADA-USD 2022-02-22 09:28:00+00:00 2022-03-22 09:26:00+00:00 (20103, 7)
XRP-USD 2022-02-22 09:28:00+00:00 2022-03-22 09:26:00+00:00 (20017, 7)
SHIB-USD 2022-02-22 09:28:00+00:00 2022-03-22 09:26:00+00:00 (20071, 7)


In [ ]:
# Building the master dataframe

series_test = coin_dataframes['ETH-USD']['Datetime'].astype(str)
dataframes = []
for name in coin_names:
  coin_dataframes[name]['Datetime'] = coin_dataframes[name]['Datetime'].astype(str)
  coin_dataframes[name][name] = coin_dataframes[name]['Close']
  coin_dataframes[name] = coin_dataframes[name][[name, 'Datetime']]
  dataframes.append(coin_dataframes[name])

from functools import reduce
full_data = reduce(lambda  left,right: pd.merge(left,right,on=['Datetime'],
                                            how='outer'), dataframes)


full_data['Datetime'] = pd.to_datetime(full_data['Datetime'])
full_data = full_data.sort_values(by='Datetime')
full_data.head()

,BTC-USD,Datetime,ETH-USD,ATOM-USD,DOGE-USD,ADA-USD,XRP-USD,SHIB-USD
0,37071.628906,2022-02-22 09:28:00+00:00,2541.579834,23.736961,0.127906,0.841923,0.689876,0.000024
1,37068.906250,2022-02-22 09:30:00+00:00,2541.201660,23.733599,0.127918,0.841909,0.689850,0.000024
2,37093.632812,2022-02-22 09:32:00+00:00,2541.557373,23.750843,0.128060,0.842479,0.690322,0.000024
3,37149.937500,2022-02-22 09:34:00+00:00,2544.920654,23.806026,0.128187,0.844554,0.691489,0.000024
4,37223.550781,2022-02-22 09:36:00+00:00,2550.543701,23.845989,0.128365,0.846691,0.693543,0.000024


### Data Preprocessing
- Fill in the null values
- Min Max Scaler
- Visualize the Data

In [ ]:
# fill null values with previous value (indicating no change)
full_data = full_data.ffill()

In [ ]:
full_data.set_index("Datetime", drop=True, inplace=True)

In [ ]:
full_data.shape

(20126, 7)

In [ ]:
# Standardize the data temporarily so that they can be displayed together
# train_df and test_df will not actually be used later. 

training_set = full_data[:20000].copy()
test_set = full_data[20000:].copy()

train_mean = training_set.mean()
train_std = training_set.std()
print(train_mean)

train_df = (test_set - train_mean) / train_std
test_df = (test_set - train_mean) / train_std


df_std = (full_data - train_mean) / train_std
df_std.head()

BTC-USD     39968.968654
ETH-USD      2708.503871
ATOM-USD       28.344236
DOGE-USD        0.121752
ADA-USD         0.855788
XRP-USD         0.757068
SHIB-USD        0.000024
dtype: float64


,BTC-USD,ETH-USD,ATOM-USD,DOGE-USD,ADA-USD,XRP-USD,SHIB-USD
Datetime,,,,,,,
2022-02-22 09:28:00+00:00,-1.529999,-1.127382,-2.068593,0.978676,-0.273813,-1.776551,0.361026
2022-02-22 09:30:00+00:00,-1.531437,-1.129936,-2.070103,0.980548,-0.274086,-1.777239,0.360221
2022-02-22 09:32:00+00:00,-1.518379,-1.127534,-2.062361,1.003149,-0.262839,-1.764766,0.372236
2022-02-22 09:34:00+00:00,-1.488646,-1.104819,-2.037584,1.023305,-0.221859,-1.733904,0.401412
2022-02-22 09:36:00+00:00,-1.449773,-1.066841,-2.019642,1.051613,-0.179652,-1.679581,0.443408


In [ ]:
# Displaying the distributions of the coin prices

import plotly.graph_objects as go

fig = go.Figure()


for coin in coin_names:
  fig.add_trace(go.Violin(y=df_std[coin],
                          name=coin,
                          box_visible=False,
                          meanline_visible=True))
fig.update_layout(
    title_text="Standardized Prices of the 7 Cryptocurrencies",
    violingap=0, violingroupgap=0, violinmode='overlay',
    xaxis_title="Coins",
    yaxis_title="Standardized Closing Price",)
fig.show()

In [ ]:
# Viewing the trends in coin prices all together.
# We can see that they tend to move together

import plotly.express as px

fig = px.area(df_std, labels={
                     "Datetime": "Date",
                     "value": "Standardized Closing Price"
                 },
                title="Standardized Coin Trends")

fig.show()

In [ ]:
# Apply a min-max scaler to the data

scaler= MinMaxScaler(feature_range=(0,1))
training_set = scaler.fit_transform(training_set)
test_set = scaler.transform(test_set)

### Creating the datasets to be passed to the model
Creating the training and testing dataset involves creating datasets that all have 100 time steps, all stored in the same vector.

In [ ]:
target_index = list(full_data.columns).index("SHIB-USD")
X_train = []
y_train = []

past_window = 100 # number of days to look behind
future_window = 1 #number of days to look in the future

for i in range(past_window, len(training_set) - future_window):
  
  y_train.append(training_set[i+1:i+1+future_window, target_index])
  X_train.append(training_set[i-past_window:i, :])

X_train, y_train = np.array(X_train), np.array(y_train)
X_train.shape, y_train.shape



((19899, 100, 7), (19899, 1))

### Building the Models
Model 0

In [ ]:
multi_lstm_model = tf.keras.Sequential([
    Bidirectional(LSTM(units=60, return_sequences=True, input_shape=(past_window, X_train.shape[2]))),
    Dropout(.2),
    LSTM(units=60, return_sequences=False), 
    Dropout(.2),

    Dense(units=1, activation='relu')
])

multi_lstm_model.compile(optimizer='adam', loss='mean_squared_error')

multi_lstm_model.fit(x = X_train, y = y_train, epochs=25, batch_size=32)

Epoch 1/25
622/622 [==============================] - 17s 15ms/step - loss: 0.0019
Epoch 2/25
622/622 [==============================] - 9s 15ms/step - loss: 9.1388e-04
Epoch 3/25
622/622 [==============================] - 9s 15ms/step - loss: 6.9069e-04
Epoch 4/25
622/622 [==============================] - 10s 15ms/step - loss: 5.7317e-04
Epoch 5/25
622/622 [==============================] - 10s 15ms/step - loss: 4.6305e-04
Epoch 6/25
622/622 [==============================] - 10s 16ms/step - loss: 4.1923e-04
Epoch 7/25
622/622 [==============================] - 10s 15ms/step - loss: 3.8268e-04
Epoch 8/25
622/622 [==============================] - 9s 15ms/step - loss: 3.5761e-04
Epoch 9/25
622/622 [==============================] - 9s 15ms/step - loss: 3.3281e-04
Epoch 10/25
622/622 [==============================] - 9s 15ms/step - loss: 3.3551e-04
Epoch 11/25
622/622 [==============================] - 10s 15ms/step - loss: 3.2623e-04
Epoch 12/25
622/622 [=============================

Model 1

In [ ]:
multi_lstm_model_1 = tf.keras.Sequential([
    Bidirectional(LSTM(units=30, return_sequences=True, input_shape=(past_window, X_train.shape[2]))),
    Dropout(.2),
    Bidirectional(LSTM(units=30, return_sequences=True)), 
    Dropout(.2),
    LSTM(units=30, return_sequences=False), 
    Dropout(.2),

    Dense(units=1, activation='relu')
])

multi_lstm_model_1.compile(optimizer='adam', loss='mean_squared_error')

multi_lstm_model_1.fit(x = X_train, y = y_train, epochs=25, batch_size=32)

Epoch 1/25
622/622 [==============================] - 23s 22ms/step - loss: 0.0049
Epoch 2/25
622/622 [==============================] - 14s 22ms/step - loss: 0.0015
Epoch 3/25
622/622 [==============================] - 14s 22ms/step - loss: 0.0010
Epoch 4/25
622/622 [==============================] - 14s 22ms/step - loss: 8.3160e-04
Epoch 5/25
622/622 [==============================] - 13s 22ms/step - loss: 7.0061e-04
Epoch 6/25
622/622 [==============================] - 14s 23ms/step - loss: 6.1841e-04
Epoch 7/25
622/622 [==============================] - 14s 23ms/step - loss: 6.1416e-04
Epoch 8/25
622/622 [==============================] - 14s 23ms/step - loss: 6.0104e-04
Epoch 9/25
622/622 [==============================] - 14s 23ms/step - loss: 5.6537e-04
Epoch 10/25
622/622 [==============================] - 13s 22ms/step - loss: 5.8535e-04
Epoch 11/25
622/622 [==============================] - 14s 22ms/step - loss: 5.7414e-04
Epoch 12/25
622/622 [==============================] 

Model 2

In [ ]:
multi_lstm_model_2 = tf.keras.Sequential([
    Bidirectional(LSTM(units=30, return_sequences=True, input_shape=(past_window, X_train.shape[2]))),
    Dropout(.2),
    Bidirectional(LSTM(units=30, return_sequences=True)), 
    Dropout(.2),
    LSTM(units=30, return_sequences=False), 
    Dropout(.2),

    Dense(units=20, activation='relu'),
    Dense(units=1, activation='relu')
])

multi_lstm_model_2.compile(optimizer='adam', loss='mean_squared_error')

multi_lstm_model_2.fit(x = X_train, y = y_train, epochs=25, batch_size=32)

Epoch 1/25
622/622 [==============================] - 23s 22ms/step - loss: 0.0030
Epoch 2/25
622/622 [==============================] - 14s 22ms/step - loss: 7.7991e-04
Epoch 3/25
622/622 [==============================] - 14s 22ms/step - loss: 5.4019e-04
Epoch 4/25
622/622 [==============================] - 14s 22ms/step - loss: 4.3866e-04
Epoch 5/25
622/622 [==============================] - 14s 22ms/step - loss: 4.0016e-04
Epoch 6/25
622/622 [==============================] - 14s 22ms/step - loss: 3.5883e-04
Epoch 7/25
622/622 [==============================] - 14s 22ms/step - loss: 3.3809e-04
Epoch 8/25
622/622 [==============================] - 14s 22ms/step - loss: 3.2226e-04
Epoch 9/25
622/622 [==============================] - 14s 22ms/step - loss: 2.9337e-04
Epoch 10/25
622/622 [==============================] - 14s 22ms/step - loss: 2.9536e-04
Epoch 11/25
622/622 [==============================] - 14s 22ms/step - loss: 2.7379e-04
Epoch 12/25
622/622 [========================

### Evaluating the Model

In [ ]:
# Prepare X-test to produce predictions
X_test = []

inputs = full_data[full_data.shape[0] - test_set.shape[0] - past_window: ]
input_index = inputs.index[100:]
inputs = scaler.transform(inputs)

for i in range(past_window, len(inputs)-future_window):
    X_test.append(inputs[i-past_window:i])

X_test = np.array(X_test)
X_test.shape

(125, 100, 7)

In [ ]:
# Y-value predictions
y_reals = []

outputs = full_data[20000:]
outputs = outputs['SHIB-USD']

for i in range(0 , len(outputs) - future_window):

  y_reals.append(outputs[i: i+future_window])

y_reals = np.array(y_reals)
y_reals.shape


(125, 1)

In [ ]:
# Undoing the Min-Max Transformation

def unscale_y(y_pred):
    t = np.zeros(shape=(len(y_pred), X_train.shape[2]))
    t[:, target_index] = y_pred
    t = scaler.inverse_transform(t)[:, target_index]
    return t

y_pred_0 = multi_lstm_model.predict(X_test)
y_pred_0 = np.array([unscale_y(i) for i in y_pred_0])

y_pred_1 = multi_lstm_model_1.predict(X_test)
y_pred_1 = np.array([unscale_y(i) for i in y_pred_1])

y_pred_2 = multi_lstm_model_2.predict(X_test)
y_pred_2 = np.array([unscale_y(i) for i in y_pred_2])


In [ ]:
# Buidling a Naive Model for Comparison
naive_predictions = full_data[19999:-2]['SHIB-USD'].values
naive_predictions = naive_predictions.reshape(125, 1)
naive_predictions.shape

(125, 1)

# Results

In [ ]:
mse_naive = (np.square(y_reals - naive_predictions)).mean()
mse_lstm_0 = (np.square(y_reals - y_pred_0)).mean()
mse_lstm_1 = (np.square(y_reals - y_pred_1)).mean()
mse_lstm_2 = (np.square(y_reals - y_pred_2)).mean()
mse_lstm_0, mse_lstm_1, mse_lstm_2, mse_naive

(1.829856597103712e-15,
 1.185950830695644e-15,
 6.341972960957398e-15,
 4.228821839630876e-16)

In [ ]:
# predicting the general direction of the movement
def predict_percent(predictions):
  correct, total = 0, 0
  for i in range(predictions.shape[0] - 1):
    if (predictions[i] < predictions[i+1] and y_reals[i] < y_reals[i +1]) or \
    (predictions[i] > predictions[i+1] and y_reals[i] > y_reals[i +1]):
      correct += 1
    total += 1
  return np.sum(correct) / np.sum(total)

accuracy_naive = predict_percent(naive_predictions)
accuracy_lstm_0 = predict_percent(y_pred_0)
accuracy_lstm_1 = predict_percent(y_pred_1)
accuracy_lstm_2 = predict_percent(y_pred_2)
accuracy_naive, accuracy_lstm_0, accuracy_lstm_1, accuracy_lstm_2


(0.6451612903225806,
 0.5645161290322581,
 0.5161290322580645,
 0.5564516129032258)

In [ ]:
# calculate r-squared
def r_squared(predictions):
  rss = np.sum(np.square(y_reals - predictions))
  tss = np.sum(np.square(y_reals - np.mean(y_reals)))
  return 1 - (rss/tss)

print(r_squared(naive_predictions))
print(r_squared(y_pred_0))
print(r_squared(y_pred_1))
print(r_squared(y_pred_2))


0.9606069916115946
0.8295422247309017
0.8895243810368049
0.409222228969944


In [ ]:
print(np.sum(np.square(y_reals - y_pred_2)))
np.sum(np.square(y_reals - np.mean(y_reals)))

7.927466201196748e-13


1.3418694118051768e-12

In [ ]:
import plotly.graph_objects as go


# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(x=input_index, y=y_pred_0.reshape(125,),
                    mode='lines',
                    name='LSTM 0: MSE 1.8e-15'))
fig.add_trace(go.Scatter(x=input_index, y=y_pred_1.reshape(125,),
                    mode='lines',
                    name='LSTM 1: MSE 1.2e-15'))
fig.add_trace(go.Scatter(x=input_index, y=y_pred_2.reshape(125,),
                    mode='lines', name='LSTM 2: MSE 6.3e-15'))
fig.add_trace(go.Scatter(x=input_index, y=y_reals.reshape(125,),
                    mode='lines+markers', name='Real Values'))

fig.update_layout(
    title_text="Predictions of 4 Models",
    xaxis_title="Date/Time",
    yaxis_title="Shiba Price",)
fig.show()


In [ ]:
binary_accuracy = [accuracy_lstm_0, accuracy_lstm_1, accuracy_lstm_2, accuracy_naive]
binary_accuracy = [round(i, 2) for i in binary_accuracy]
r_squareds = [r_squared(y_pred_0), r_squared(y_pred_1), r_squared(y_pred_2), r_squared(naive_predictions)]
r_squareds = [round(i, 2) for i in r_squareds]
fig = go.Figure(data=[
                      go.Bar(name = 'R-Squared', x=['LSTM 0', 'LSTM 1', 'LSTM 2', 'Naive Predictions'], y=r_squareds, text=r_squareds),
                      go.Bar(name = 'Binary Accuracy', x=['LSTM 0', 'LSTM 1', 'LSTM 2', 'Naive Predictions'], y=binary_accuracy, text=binary_accuracy)])
fig.update_layout(barmode='group', title_text='Accuracy for 4 Models', yaxis_title='Percent')
fig.show()

# Conclusion
While the LSTMS seem able to track the true prices in general, they are not able to outperform the naive models. Indeed, many people believe that for processes with so much noise, like currency prices, simpler models may perform best. 

Also, the LSTM with 2 Bidirectional, and just one dense output layer seems to perform best. It may be the case that making the model deeper and deeper may in fact improve accuracy over time. 

However, there are also other methods that can be attempted for predicting crytpocurrency prices. First, an LSTM may put too much weight on the past-- only the most recent prices may matter. Second, an LSTM doesn't fully capture complex autoregressive behavior. 

It also may be true that the Shiba Inu coin is simply independent of the other coin prices. Purchases of Shiba Inu may not be heavily invested in other coins. It may be a reasonable strategy to see if other coins have had better performance.